This notebook is to batch process adaptations and analyse results

In [10]:
# Imports
import configparser
from pathlib import Path
import pathlib
from ci_adapt_utilities import *
import pickle
from tqdm.notebook import tqdm

In [11]:
# Load configuration with ini file (created running config.py)
config_file=r'C:\repos\ci_adapt\config_ci_adapt.ini'
config = configparser.ConfigParser()
config.read(config_file)

# Define paths
asset_data = config.get('DEFAULT', 'asset_data')
data_path = Path(pathlib.Path.home().parts[0]) / 'Data'
interim_data_path = data_path / 'interim' / 'collected_flood_runs'

# Define costs for different transport modes
average_train_load_tons = (896+1344+2160+1344+896+896+1344+1512+896+390)/10 # in Tons per train. Source: Kennisinstituut voor Mobiliteitsbeleid. 2023. Cost Figures for Freight Transport – final report
average_train_cost_per_ton_km = (0.014+0.018+0.047+0.045)/4 # in Euros per ton per km. Source: Kennisinstituut voor Mobiliteitsbeleid. 2023. Cost Figures for Freight Transport – final report
average_road_cost_per_ton_km = (0.395+0.375+0.246+0.203+0.138+0.153+0.125+0.103+0.122+0.099)/10 # in Euros per ton per km. Source: Kennisinstituut voor Mobiliteitsbeleid. 2023. Cost Figures for Freight Transport – final report

# Define dictionaries of return periods and adaptation unit costs
return_period_dict = {'_H_': 10,'_M_': 100,'_L_': 200}
adaptation_unit_costs = {'fwall': 7408, #considering floodwall in Germany
                         'viaduct': 36666, #considering viaduct cost
                         'bridge': 40102}  #considering bridge of 10m deck width
rp_spec_priority = set_rp_priorities(return_period_dict)

In [12]:
# Load data from baseline impact assessment
assets_path = data_path / asset_data
assets=preprocess_assets(assets_path)

# Add buffer to assets to do area intersect and create dictionaries for quicker lookup
# buffered_assets = ds.buffer_assets(assets)
geom_dict = assets['geometry'].to_dict()

print(f"{len(assets)} assets loaded.")

shortest_paths = pickle.load(open(data_path / 'interim' / 'indirect_damages' / 'shortest_paths.pkl', 'rb'))
disrupted_edges_by_basin = pickle.load(open(data_path / 'interim' / 'indirect_damages' / 'disrupted_edges_by_basin.pkl', 'rb'))
graph_r0 = pickle.load(open(data_path / 'interim' / 'indirect_damages' / 'graph_0.pkl', 'rb'))
disrupted_shortest_paths = pickle.load(open(data_path / 'interim' / 'indirect_damages' / 'disrupted_shortest_paths.pkl', 'rb'))
event_impacts = pickle.load(open(data_path / 'interim' / 'indirect_damages' / 'event_impacts.pkl', 'rb'))
full_flood_event=pickle.load(open(data_path / 'interim' / 'indirect_damages' / 'full_flood_event.pkl', 'rb'))
all_disrupted_edges = pickle.load(open(data_path / 'interim' / 'indirect_damages' / 'all_disrupted_edges.pkl', 'rb'))
collect_output = pickle.load(open(data_path / 'interim' / 'collected_flood_runs' / 'sample_collected_run.pkl', 'rb'))
print('Loaded data from baseline impact assessment')
graph_v0=create_virtual_graph(graph_r0)
graph_v=graph_v0.copy()

47210 assets loaded.
Loaded data from baseline impact assessment
Creating virtual graph...
Success: only int type values


In [13]:
adaptations={}
adaptations['baseline'] = {'l1_l2_adapt_path': None, 'added_links':[], 'l4_adapt_path': None}
adaptations['l1_trib'] = {'l1_l2_adapt_path': Path(r'C:\Data\input\adaptations\l1_tributary.geojson'), 'added_links':[], 'l4_adapt_path': None}
adaptations['l1_trib_M'] = {'l1_l2_adapt_path': Path(r'C:\Data\input\adaptations\l1_tributary_M.geojson'), 'added_links':[], 'l4_adapt_path': None}
adaptations['l1_trib_H'] = {'l1_l2_adapt_path': Path(r'C:\Data\input\adaptations\l1_tributary_H.geojson'), 'added_links':[], 'l4_adapt_path': None}
adaptations['l2_trib'] = {'l1_l2_adapt_path': Path(r'C:\Data\input\adaptations\l2_tributary.geojson'), 'added_links':[], 'l4_adapt_path': None}
adaptations['l3_trib'] = {'l1_l2_adapt_path': None, 'added_links':[(4424116, 219651487), (219651487, 111997047)], 'l4_adapt_path': None}
adaptations['l3a_trib'] = {'l1_l2_adapt_path': None, 'added_links':[(4424116, 219651487)], 'l4_adapt_path': None}
adaptations['l3b_trib'] = {'l1_l2_adapt_path': None, 'added_links':[(219651487, 111997047)], 'l4_adapt_path': None}
adaptations['l4_trib'] = {'l1_l2_adapt_path': None, 'added_links':[], 'l4_adapt_path': Path(r'C:\Data\input\adaptations\l4_tributary.geojson')}
adaptations['l1_rhine'] = {'l1_l2_adapt_path': Path(r'C:\Data\input\adaptations\l1_rhine.geojson'), 'added_links':[], 'l4_adapt_path': None}
adaptations['l1_rhine_M'] = {'l1_l2_adapt_path': Path(r'C:\Data\input\adaptations\l1_rhine_M.geojson'), 'added_links':[], 'l4_adapt_path': None}
adaptations['l1_rhine_H'] = {'l1_l2_adapt_path': Path(r'C:\Data\input\adaptations\l1_rhine_H.geojson'), 'added_links':[], 'l4_adapt_path': None}
adaptations['l2_rhine'] = {'l1_l2_adapt_path': Path(r'C:\Data\input\adaptations\l2_rhine.geojson'), 'added_links':[], 'l4_adapt_path': None}
adaptations['l3_rhine'] = {'l1_l2_adapt_path': None, 'added_links':[(112044105, 110947346)], 'l4_adapt_path': None} 
# adaptations['l3_rhine'] = {'l1_l2_adapt_path': None, 'added_links':[(112044105, 35861458)], 'l4_adapt_path': None} #35861458 instead of 110947346
adaptations['l4_rhine'] = {'l1_l2_adapt_path': None, 'added_links':[], 'l4_adapt_path': Path(r'C:\Data\input\adaptations\l4_rhine.geojson')}

In [14]:
# Define empty dictionaries to store adaptation results
direct_damages_adapted_dict = {}
indirect_damages_adapted_dict = {}
indirect_damages_adapted_full_dict = {}
adaptation_costs={}
adapted_assets_dict = {}

# Print adaptations that will be run
print(f"Processing {len(adaptations)} scenarios:")
for adapt_id in adaptations.keys():
    print('- ',adapt_id)

for adapt_id in tqdm(adaptations.keys(), desc='Adaptation runs', total=len(adaptations)):
    adaptations_df_path = data_path / 'interim' / 'adaptations' / f'{adapt_id}_adaptations.csv'

    if adaptations_df_path.exists():
        print(f"Adaptation {adapt_id} already processed. Skipping.")
        continue

    # reset variables
    graph_v=graph_v0.copy()

    # Load adaptations dictionary to the relevant variables
    l1_l2_adapt_path = adaptations[adapt_id]['l1_l2_adapt_path']
    added_links = adaptations[adapt_id]['added_links']  
    l4_adapt_path = adaptations[adapt_id]['l4_adapt_path']

    # Load adaptation data
    if l1_l2_adapt_path is not None:
        adapted_area = gpd.read_file(l1_l2_adapt_path).to_crs(3857)
    else:
        adapted_area = None
    if l4_adapt_path is not None:
        adapted_route_area = gpd.read_file(l4_adapt_path).to_crs(3857)
    else:
        adapted_route_area = None

    # Apply adaptations
    adapted_assets, adaptations_df, demand_reduction_dict, l3_adaptation_costs = apply_adaptations(adapted_area, assets, collect_output, interim_data_path, rp_spec_priority, adaptation_unit_costs, shortest_paths, graph_v, added_links, adapted_route_area)

    # Calculate l1 adaptation costs
    local_haz_path=data_path/r'Floods\Germany\fluvial_undefended\raw_subsample\validated_geometries'
    l1_adaptation_costs=calculate_l1_costs(local_haz_path, interim_data_path, adapted_area, adaptation_unit_costs, adapted_assets) 

    # Run adapted damages for individual hazard maps
    direct_damages_adapted, indirect_damages_adapted, adaptation_run_full, l2_adaptation_costs, overlay_assets_lists = run_adapted_damages(collect_output, disrupted_edges_by_basin, interim_data_path, assets, geom_dict, adapted_assets, adaptations_df, rp_spec_priority, adaptation_unit_costs, shortest_paths, graph_v, average_train_load_tons, average_train_cost_per_ton_km, average_road_cost_per_ton_km, demand_reduction_dict)

    # Run adapted damages for full flood event
    indirect_damages_adapted_full = calculate_indirect_dmgs_fullflood(full_flood_event, overlay_assets_lists, adaptation_run_full, assets, all_disrupted_edges, shortest_paths, graph_v, average_train_load_tons, average_train_cost_per_ton_km, average_road_cost_per_ton_km, demand_reduction_dict)


    # Fill in missing values in dictionaries
    for hazard_map in collect_output.keys():
        if direct_damages_adapted[hazard_map]=={}:
            direct_damages_adapted[hazard_map]=collect_output[hazard_map]
        if indirect_damages_adapted[hazard_map]=={}:
            indirect_damages_adapted[hazard_map]=event_impacts[hazard_map] if hazard_map in event_impacts.keys() else 0.0
    
    # Store results in dictionaries
    direct_damages_adapted_dict[adapt_id] = direct_damages_adapted
    indirect_damages_adapted_dict[adapt_id] = indirect_damages_adapted
    indirect_damages_adapted_full_dict[adapt_id] = indirect_damages_adapted_full
    adapted_assets_dict[adapt_id] = adapted_assets
    adaptation_costs[adapt_id] = {'l1': l1_adaptation_costs, 'l2': l2_adaptation_costs, 'l3': l3_adaptation_costs}
    adaptations_df.to_csv(data_path / 'interim' / 'adaptations' / f'{adapt_id}_adaptations.csv')

    # break  

Processing 15 scenarios:
-  baseline
-  l1_trib
-  l1_trib_M
-  l1_trib_H
-  l2_trib
-  l3_trib
-  l3a_trib
-  l3b_trib
-  l4_trib
-  l1_rhine
-  l1_rhine_M
-  l1_rhine_H
-  l2_rhine
-  l3_rhine
-  l4_rhine


Adaptation runs:   0%|          | 0/15 [00:00<?, ?it/s]

Adaptation baseline already processed. Skipping.
Adaptation l1_trib already processed. Skipping.
Adaptation l1_trib_M already processed. Skipping.
Adaptation l1_trib_H already processed. Skipping.
Adaptation l2_trib already processed. Skipping.
Adaptation l3_trib already processed. Skipping.
Adaptation l3a_trib already processed. Skipping.
Adaptation l3b_trib already processed. Skipping.
Adaptation l4_trib already processed. Skipping.
Adaptation l1_rhine already processed. Skipping.
Adaptation l1_rhine_M already processed. Skipping.
Adaptation l1_rhine_H already processed. Skipping.
Adaptation l2_rhine already processed. Skipping.
Adaptation l3_rhine already processed. Skipping.
Applying adaptation: reduced demand for routes:  [('node_15695', 'node_15775'), ('node_15775', 'node_15695'), ('node_16075', 'node_15695'), ('node_15695', 'node_16075'), ('node_35598', 'node_15695'), ('node_15695', 'node_35598'), ('node_15695', 'node_8204'), ('node_8204', 'node_15695'), ('node_18308', 'node_156

Processing adapted damages by hazard map:   0%|          | 0/183 [00:00<?, ?it/s]

Processing full flood events:   0%|          | 0/3 [00:00<?, ?it/s]

In [15]:
# report output dataframe
output_df = pd.DataFrame.from_dict([direct_damages_adapted_dict, indirect_damages_adapted_dict, indirect_damages_adapted_full_dict, adapted_assets_dict, adaptation_costs])
output_df

,l4_rhine
0,"{'flood_DERP_RW_H_4326_2080410170': {2736: (0,..."
1,{'flood_DERP_RW_H_4326_2080410170': 467036.273...
2,"{'flood_DERP_RW_H': 55050826.584263116, 'flood..."
3,"Empty GeoDataFrame Columns: [osm_id, asset, na..."
4,"{'l1': None, 'l2': {}, 'l3': {}}"


In [16]:
# Save results
# adapt_id= 'l1_trib_test'
for adapt_id in adaptations.keys():
    if not adapt_id in direct_damages_adapted_dict.keys():
        continue
    direct_damages_adapted_path = data_path / 'output' / f'adapted_direct_damages_{adapt_id}.pkl'
    indirect_damages_adapted_path = data_path / 'output' / f'adapted_indirect_damages_{adapt_id}.pkl'
    indirect_damages_adapted_full_path = data_path / 'output' / f'adapted_indirect_damages_full_{adapt_id}.pkl'
    # adaptations_df_path = data_path / 'output' / f'adaptations_{adapt_id}.csv'
    adapted_assets_path = data_path / 'output' / f'adapted_assets_{adapt_id}.pkl'
    adaptation_costs_path = data_path / 'output' / f'adaptation_costs_{adapt_id}.pkl'

    with open(direct_damages_adapted_path, 'wb') as f:
        pickle.dump(direct_damages_adapted_dict[adapt_id], f)
    with open(indirect_damages_adapted_path, 'wb') as f:
        pickle.dump(indirect_damages_adapted_dict[adapt_id], f)
    with open(indirect_damages_adapted_full_path, 'wb') as f:
        pickle.dump(indirect_damages_adapted_full_dict[adapt_id], f)    
    with open(adapted_assets_path, 'wb') as f:
        pickle.dump(adapted_assets_dict[adapt_id], f)
    with open(adaptation_costs_path, 'wb') as f:
        pickle.dump(adaptation_costs[adapt_id], f)
    print(f'Saved results for adaptation: {adapt_id}')
        


Saved results for adaptation: l4_rhine


In [ ]:
    
# direct_damages_adapted_path = data_path / 'output' / f'adapted_direct_damages_{adapt_id}.pkl'
# indirect_damages_adapted_path = data_path / 'output' / f'adapted_indirect_damages_{adapt_id}.pkl'
# indirect_damages_adapted_full_path = data_path / 'output' / f'adapted_indirect_damages_full_{adapt_id}.pkl'
# adaptations_df_path = data_path / 'output' / f'adaptations_{adapt_id}.csv'
# adapted_assets_path = data_path / 'output' / f'adapted_assets_{adapt_id}.pkl'


# with open(direct_damages_adapted_path, 'wb') as f:
#     pickle.dump(direct_damages_adapted, f)
# with open(indirect_damages_adapted_path, 'wb') as f:
#     pickle.dump(indirect_damages_adapted, f)
# with open(indirect_damages_adapted_full_path, 'wb') as f:    
#     pickle.dump(indirect_damages_adapted_full, f)
# adaptations_df.to_csv(adaptations_df_path, index=False)

# l3_geometries = {}
# if added_links != [] and l1_l2_adapt_path is None and l4_adapt_path is None:
#     for u, v, k, attr in graph_v.edges(keys=True, data=True):
#         if 'osm_id' not in attr:
#             continue
#         if 'l3_adaptation' in attr['osm_id']:
#             # Ensure the geometry is a valid Shapely geometry object
#             geometry = attr['geometry']
#             if isinstance(geometry, list) and len(geometry) == 1 and isinstance(geometry[0], shapely.LineString):
#                 geometry = geometry[0]
#             if isinstance(geometry, shapely.LineString):
#                 l3_geometries[(u, v)] = geometry
#             else:
#                 print(f"Invalid geometry for edge ({u}, {v}): {geometry}")

#     gdf_l3_edges = gpd.GeoDataFrame.from_dict(l3_geometries, orient='index', columns=['geometry'], geometry='geometry', crs=3857)
#     gdf_l3_edges.reset_index(inplace=True)

#     gdf_l3_edges = gpd.GeoDataFrame(list(l3_geometries.items()), columns=['edge', 'geometry'], geometry='geometry', crs=3857)
#     with open(adapted_assets_path, 'wb') as f:
#         pickle.dump(gdf_l3_edges, f)

# else:
#     with open(adapted_assets_path, 'wb') as f:
#         pickle.dump(adapted_assets, f)



In [ ]:


direct_damages_baseline_sum = {
    key: (
        sum(v[0] for v in collect_output[key].values()), 
        sum(v[1] for v in collect_output[key].values())
        ) 
        for key in collect_output if key in collect_output.keys()
        }
direct_damages_adapted_sum = {
    key: (
        sum(v[0] for v in direct_damages_adapted[key].values()), 
        sum(v[1] for v in direct_damages_adapted[key].values())
        ) 
        for key in direct_damages_adapted if key in direct_damages_adapted.keys()
        }

direct_damages_diff = {
    key: (
        direct_damages_baseline_sum[key][0] - direct_damages_adapted_sum[key][0], 
        direct_damages_baseline_sum[key][1] - direct_damages_adapted_sum[key][1]
        ) for key in direct_damages_baseline_sum
        }

direct_damages_diff

Visualisations
pd.options.display.float_format = "{:,.2f}".format

total_damages_adapted={}
for hazard_map in direct_damages_adapted.keys():

    map_rp_spec = hazard_map.split('_')[-3]

    adap_costs=adaptations_df['adaptation_cost']
    summed_adaptation_costs = sum(adap_costs)

    #direct damages
    dd_bl=direct_damages_adapted[hazard_map][0]
    summed_dd_bl_lower=sum([v[0] for v in dd_bl.values()])
    summed_dd_bl_upper=sum([v[1] for v in dd_bl.values()])
    dd_ad=direct_damages_adapted[hazard_map][1]
    summed_dd_ad_lower=sum([v[0] for v in dd_ad.values()])
    summed_dd_ad_upper=sum([v[1] for v in dd_ad.values()])
    
    #indirect damages
    if hazard_map not in event_impacts.keys():
        print(f'{hazard_map} not in event_impacts')
        id_bl=0
        id_ad=0
        id_ad_cleaned=0
    else:
        id_bl=event_impacts[hazard_map]
        id_ad=indirect_damages_adapted[hazard_map]
        id_ad_cleaned = 0 if id_ad == 99999999999999 else id_ad

    total_damages_adapted[hazard_map]=(map_rp_spec, summed_adaptation_costs, (summed_dd_bl_lower, summed_dd_bl_upper), (summed_dd_ad_lower, summed_dd_ad_upper), id_bl, id_ad_cleaned)
    
total_damages_adapted_df=pd.DataFrame(total_damages_adapted)
total_damages_adapted_df=total_damages_adapted_df.T
total_damages_adapted_df.columns=['return_period','summed_adaptation_costs', 'summed_dd_bl', 'summed_dd_ad', 'indirect damage baseline [€]', 'indirect damage adapted [€]']

# round and turn to million euros for reporting
total_damages_adapted_df_mill=total_damages_adapted_df.copy()
total_damages_adapted_df_mill['summed_adaptation_costs [M€]']=total_damages_adapted_df_mill['summed_adaptation_costs']/1e6
total_damages_adapted_df_mill['summed_dd_bl [M€]']=total_damages_adapted_df_mill['summed_dd_bl'].apply(lambda x: (x[0]/1e6, x[1]/1e6))
total_damages_adapted_df_mill['summed_dd_ad [M€]']=total_damages_adapted_df_mill['summed_dd_ad'].apply(lambda x: (x[0]/1e6, x[1]/1e6))
total_damages_adapted_df_mill['indirect damage baseline [M€]']=total_damages_adapted_df_mill['indirect damage baseline [€]']/1e6
total_damages_adapted_df_mill['indirect damage adapted [M€]']=total_damages_adapted_df_mill['indirect damage adapted [€]']/1e6
total_damages_adapted_df_mill.drop(['summed_adaptation_costs','summed_dd_bl', 'summed_dd_ad', 'indirect damage baseline [€]', 'indirect damage adapted [€]'], axis=1, inplace=True)



custom_order = ['H', 'M', 'L', 'Unknown']

total_damages_adapted_df_mill['return_period'] = pd.Categorical(total_damages_adapted_df_mill['return_period'], 
                                                                categories=custom_order, ordered=True)
sorted_total_damages_adapted_df_mill = total_damages_adapted_df_mill.sort_values(by='return_period', ascending=True)
sorted_total_damages_adapted_df_mill
shortest_paths_assets={}
od_assets=[]
o_geoms=[]
d_geoms=[]
for (o,d), (path, demand) in shortest_paths.items():
    od_assets_by_sp=[]
    o_geoms.append(graph_v.nodes[o]['geometry'])
    d_geoms.append(graph_v.nodes[d]['geometry'])

    for i in range(len(path)-1):
        x=graph_v.edges[path[i], path[i+1], 0]
        od_assets_by_sp.append(x['osm_id'])
        od_assets.append(x['osm_id'])

    shortest_paths_assets[(o,d)]=(od_assets_by_sp, demand)
assets_sps=assets.loc[assets['osm_id'].isin(set(od_assets))].copy()


# Repeat for shortest paths under adapted conditions
shortest_paths_adapted_assets={}
for flood_map, od_dict in disrupted_shortest_paths.items():
    for (o,d), (path, demand) in od_dict.items():
        od_assets_by_sp_adapted=[]
        for i in range(len(path)-1):
            x=graph_v.edges[path[i], path[i+1], 0]
            od_assets_by_sp_adapted.append(x['osm_id'])
        shortest_paths_adapted_assets[flood_map, (o,d)]=(od_assets_by_sp_adapted, demand)


df_shortest_paths=pd.DataFrame(shortest_paths_assets).T
df_shortest_paths.columns=['path', 'demand']
df_shortest_paths


df_shortest_paths_adapted=pd.DataFrame(shortest_paths_adapted_assets).T
df_shortest_paths_adapted.columns=['path', 'demand']
df_shortest_paths_adapted




# EXPAND FOR VISUALISATION SCRIPT
# Create visualisation for the basin and the discharge points
from lonboard import Map, PolygonLayer, PathLayer, BaseLayer, ScatterplotLayer
import ast
# MIRACA color scheme
color_string = config.get('DEFAULT', 'miraca_colors')
miraca_colors = ast.literal_eval(color_string)

basin_id = 2080430320
rp_vis = 'M'
overlay_assets = load_baseline_run(f'flood_DERP_RW_{rp_vis}_4326_{basin_id}', interim_data_path, only_overlay=True)
    
# Set path for basin to add to visualization
basin_path = rf'C:\Data\Floods\basins\hybas_eu_{int(basin_id)}.shp'

# Generate the basin layer
basin = gpd.read_file(basin_path)
layer_basin = PolygonLayer.from_geopandas(basin,
    get_fill_color=miraca_colors['grey_100'],
    get_line_color=miraca_colors['accent green'], get_line_width=10,
    auto_highlight=False,
    filled=True, opacity=0.1)

# Generate od layer for visualization
od_geoms=o_geoms+d_geoms
od_geoms_gdf=gpd.GeoDataFrame(geometry=od_geoms).set_crs(3857)
layer_od = ScatterplotLayer.from_geopandas(od_geoms_gdf, get_fill_color=miraca_colors['red_danger'], get_radius=200, opacity=0.8, auto_highlight=True)



# Set path for the protected area to add to visualization
adapt_path = l1_l2_adapt_path
if adapt_path is not None:
    adapt_area = gpd.read_file(adapt_path)
    if len(adapt_area.adapt_level.unique())==1 and adapt_area.adapt_level.unique()[0]==1:
        layer_adapted_area = PolygonLayer.from_geopandas(adapt_area,
                                                         get_fill_color=miraca_colors['green_800'],
                                                         get_line_color=miraca_colors['primary blue'], get_line_width=10,
                                                         auto_highlight=False, filled=True, opacity=0.1)
    else:
        layer_adapted_area = None
else:
    layer_adapted_area = None

# Create layer for assets for visualization
layer_assets = PathLayer.from_geopandas(assets.drop(columns=['buffered', 'other_tags']), get_width=80, get_color=miraca_colors['grey_400'], auto_highlight=True, )
layer_shortest_path_assets = PathLayer.from_geopandas(assets_sps.drop(columns=['buffered', 'other_tags']), get_width=80, get_color=miraca_colors['black'], auto_highlight=True)
affected_assets = [asset_id for asset_id in list(set(overlay_assets.asset.values))]
layer_affected_assets = PathLayer.from_geopandas(assets.iloc[affected_assets].drop(columns=['buffered', 'other_tags']), get_width=80, get_color=miraca_colors['red_danger'], auto_highlight=True)

try:
    layer_protected_assets = PathLayer.from_geopandas(adapted_assets.drop(columns=['buffered', 'other_tags']), get_width=90, get_color=miraca_colors['green_success'], auto_highlight=True)
except:
    layer_protected_assets = None
try:
    layer_l3_edges = PathLayer.from_geopandas(gdf_l3_edges, get_width=90, get_color=miraca_colors['green_success'])
except:
    layer_l3_edges = None
layer_assets_raw = [layer_assets, layer_shortest_path_assets, layer_affected_assets, layer_protected_assets, layer_l3_edges]
layers_assets = [layer for layer in layer_assets_raw if layer is not None]

# Generate flood layers and protection layers for visualization
flood_plot_path=rf'Floods\Germany\basin_intersections\DERP_RW_{rp_vis}_4326_hybas_intersections\flood_DERP_RW_{rp_vis}_4326_{basin_id}.geojson'
flood_m = data_path / flood_plot_path
flood_gdf=gpd.read_file(flood_m)
layers_flood=[]
f_area_colors = {1:'blue', 3:'green'}
for f_area in flood_gdf.flood_area.unique():
    for f_depth in flood_gdf.depth_class.unique():
        subset_gdf = flood_gdf[(flood_gdf.depth_class==f_depth) & (flood_gdf.flood_area==f_area)]
        if not subset_gdf.empty:
            color_key=f'{f_area_colors[f_area]}_{f_depth}00'
            layers_flood.append(PolygonLayer.from_geopandas(subset_gdf, 
                                                            get_fill_color=miraca_colors[color_key], 
                                                            opacity=0.5, 
                                                            stroked=False))

layers=[]
if layer_basin is not None:
    layers.append(layer_basin)
else:
    print('No basin layer')

if layer_adapted_area is not None:
    layers.append(layer_adapted_area)
else:
    print('No protected area layer')

if layers_flood is not None:
    layers.extend(layers_flood)
else:
    print('No flood layers')

if layer_assets is not None:
    layers.extend(layers_assets)
else:
    print('No asset layer')
Voyager = 'https://basemaps.cartocdn.com/gl/voyager-gl-style/style.json'

if layer_od is not None:
    layers.append(layer_od)
else:
    print('No od layer')
m = Map(layers, show_tooltip=True, basemap_style=Voyager, view_state={"longitude": 7.91, "latitude": 49.91, "zoom": 11})

m
plot_assets=assets.drop(columns=['buffered', 'other_tags']).to_crs(epsg=4326)
plot_assets.head(3)
import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
import geopandas as gpd

# helper function
def zoomlevel_from_deg(deg): #https://stackoverflow.com/questions/30052990/how-to-use-openstreetmap-background-on-matplotlib-basemap
    "Calculate OSM zoom level from a span in degrees.  Adjust +/-1 as desired"
    from numpy import log2, clip, floor
    zoomlevel = int(clip(floor(log2(360) - log2(delta)),0,20 ))
    return zoomlevel 

# Convert basin to EPSG:4326 and get the total bounds
extent = basin.to_crs(epsg=4326).total_bounds

# Add clearance around the basin
clearance = 0.5  # 10% clearance around the basin bounds
lon_min, lat_min, lon_max, lat_max = extent
lon_range = lon_max - lon_min
lat_range = lat_max - lat_min

lon_min -= clearance * lon_range
lat_min -= clearance * lat_range
lon_max += clearance * lon_range
lat_max += clearance * lat_range

# Calculate aspect ratio
aspect_ratio = lon_range / lat_range

lon_i = lon_min + (lon_max - lon_min) / 2
lat_i = lat_min + (lat_max - lat_min) / 2
delta = 0.0016 # 0.0012 # 38 to 0.0002 degrees to avoid north pole
zoom = zoomlevel_from_deg(delta)-5 # 10 #  0-19 
print(f"Zoom Level: {zoom}")

# Create a plot with the OSM tiles
request_osm = cimgt.OSM()

fig = plt.figure(figsize=(10 * aspect_ratio, 10))
ax = plt.axes(projection=request_osm.crs)
ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())
ax.add_image(request_osm, zoom, alpha=0.5)  # Adjust zoom level as needed

# Plot the basin
basin.to_crs(epsg=4326).plot(ax=ax, facecolor='none', edgecolor='grey', linewidth=1, transform=ccrs.PlateCarree())

# Plot the assets
plot_assets.plot(ax=ax, color='grey', markersize=10, transform=ccrs.PlateCarree())

# Plot the flood areas
for f_area in flood_gdf.flood_area.unique():
    for f_depth in flood_gdf.depth_class.unique():
        subset_gdf = flood_gdf[(flood_gdf.depth_class==f_depth) & (flood_gdf.flood_area==f_area)]
        if not subset_gdf.empty:
            color_key=f'{f_area_colors[f_area]}_{f_depth}00'
            subset_gdf.to_crs(epsg=4326).plot(ax=ax, facecolor=miraca_colors[color_key], edgecolor=None, linewidth=2, alpha=0.5, transform=ccrs.PlateCarree())

# Plot the shortest paths
plot_assets_sps = assets_sps.drop(columns=['buffered', 'other_tags']).to_crs(epsg=4326)
plot_assets_sps.plot(ax=ax, color='black', markersize=10, transform=ccrs.PlateCarree())

# Plot the affected assets
plot_assets_affected = assets.iloc[affected_assets].drop(columns=['buffered', 'other_tags']).to_crs(epsg=4326)
plot_assets_affected.plot(ax=ax, color='red', markersize=10, transform=ccrs.PlateCarree())

# Plot the OD points
od_geoms_gdf.to_crs(epsg=4326).plot(ax=ax, color='blue', markersize=10, transform=ccrs.PlateCarree())

ax.gridlines(draw_labels=True)
plt.show()
import cartopy
# Convert basin to EPSG:4326 and get the total bounds
extent_new = assets.to_crs(epsg=4326).total_bounds

# Add clearance around the basin
clearance = 0  # 10% clearance around the basin bounds

lon_min_new, lat_min_new, lon_max_new, lat_max_new = extent_new
lon_range_new = lon_max_new - lon_min_new
lat_range_new = lat_max_new - lat_min_new

lon_min_new -= clearance * lon_range_new
lat_min_new -= clearance * lat_range_new
lon_max_new += clearance * lon_range_new
lat_max_new += clearance * lat_range_new

# Calculate aspect ratio
aspect_ratio_new = lon_range_new / lat_range_new

lon_i_new = plot_assets.total_bounds[0] + (plot_assets.total_bounds[2] - plot_assets.total_bounds[0]) / 2
lat_i_new = plot_assets.total_bounds[1] + (plot_assets.total_bounds[3] - plot_assets.total_bounds[1]) / 2
delta_new = 0.0016 # 0.0012 # 38 to 0.0002 degrees to avoid north pole
zoom_new = zoomlevel_from_deg(delta_new)-5 # 10 #  0-19
print(f"Zoom Level: {zoom_new}")

fig = plt.figure(figsize=(10 * aspect_ratio_new, 10))
ax = plt.axes(projection=request_osm.crs)
ax.set_extent([lon_min_new, lon_max_new, lat_min_new, lat_max_new], crs=ccrs.PlateCarree())
ax.add_image(request_osm, zoom_new, alpha=0.5)  # Adjust zoom level as needed

# Plot the basin
basin.to_crs(epsg=4326).plot(ax=ax, facecolor='none', edgecolor='grey', linewidth=1, transform=ccrs.PlateCarree())

# Plot the assets
plot_assets.plot(ax=ax, color='grey', markersize=10, transform=ccrs.PlateCarree())

# Plot the flood areas
for f_area in flood_gdf.flood_area.unique():
    for f_depth in flood_gdf.depth_class.unique():
        subset_gdf = flood_gdf[(flood_gdf.depth_class==f_depth) & (flood_gdf.flood_area==f_area)]
        if not subset_gdf.empty:
            color_key=f'{f_area_colors[f_area]}_{f_depth}00'
            subset_gdf.to_crs(epsg=4326).plot(ax=ax, facecolor=miraca_colors[color_key], edgecolor=None, linewidth=2, alpha=0.5, transform=ccrs.PlateCarree())

# Plot the shortest paths
plot_assets_sps = assets_sps.drop(columns=['buffered', 'other_tags']).to_crs(epsg=4326)
plot_assets_sps.plot(ax=ax, color='black', markersize=10, transform=ccrs.PlateCarree())

# Plot the affected assets
plot_assets_affected = assets.iloc[affected_assets].drop(columns=['buffered', 'other_tags']).to_crs(epsg=4326)
plot_assets_affected.plot(ax=ax, color='red', markersize=10, transform=ccrs.PlateCarree())

# Plot the OD points
od_geoms_gdf.to_crs(epsg=4326).plot(ax=ax, color='cyan', markersize=10, transform=ccrs.PlateCarree())

ax.gridlines(draw_labels=True)

# add administrative boundaries
ax.add_feature(cartopy.feature.BORDERS, linestyle=':')
plt.show()
#!pip install cartopy
import cartopy.crs as ccrs
import cartopy
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import pyproj

# Define the transformer to convert from EPSG:3857 to EPSG:4326
transformer = pyproj.Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)

# Convert coordinates of o_geoms and d_geoms to EPSG:4326
o_geoms_4326 = [transformer.transform(geom.x, geom.y) for geom in o_geoms]
d_geoms_4326 = [transformer.transform(geom.x, geom.y) for geom in d_geoms]

# Create a map with the basin and the origins and destinations
fig = plt.figure(figsize=(20, 20))
ax = plt.axes(projection=ccrs.PlateCarree())

# Plot the basin (assuming basin is in EPSG:3857)
# If basin is in EPSG:3857, convert it to EPSG:4326
basin = basin.to_crs(epsg=4326)
basin.plot(ax=ax, facecolor='none', edgecolor='grey', linewidth=2, alpha = 0.5)

# Plot the assets
plot_assets.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=2)

# Plot the flood areas
flood_gdf.plot(ax=ax, facecolor='none', edgecolor='blue', linewidth=1)

# Plot the protected area
# adapted_area.plot(ax=ax, facecolor='none', edgecolor='green', linewidth=2)

# Plot the shortest paths  
for (o,d), (path, demand) in shortest_paths.items():
    path_coords = [graph_v.nodes[node]['geometry'] for node in path]
    ax.plot([geom.x for geom in path_coords], [geom.y for geom in path_coords], 'b-', linewidth=2)

# Set the extent of the map based on basin boundaries
lon_min, lat_min, lon_max, lat_max = basin.total_bounds


# Plot the origins and destinations
for lon, lat in o_geoms_4326:
    ax.plot(lon, lat, 'ro', markersize=10)
for lon, lat in d_geoms_4326:
    ax.plot(lon, lat, 'ro', markersize=10)


# Calculate the range for longitude and latitude
lon_range = lon_max - lon_min
lat_range = lat_max - lat_min
# Add clearance around the basin
clearance = 0.6  # that is 60% of the range between min and max lon and lat on either side
# Apply clearance
lon_min -= clearance * lon_range
lat_min -= clearance * lat_range
lon_max += clearance * lon_range
lat_max += clearance * lat_range


ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())

# Add basemap
ax.add_feature(cartopy.feature.LAND, edgecolor='black')
ax.add_feature(cartopy.feature.BORDERS, linestyle=':')
ax.add_feature(cartopy.feature.LAKES, edgecolor='black')
ax.add_feature(cartopy.feature.RIVERS)    

# Add states and provinces
states_provinces = cartopy.feature.NaturalEarthFeature(
            category='cultural',  name='admin_1_states_provinces',
            scale='10m', facecolor='none')
ax.add_feature(states_provinces, edgecolor='black', zorder=10, linestyle = '-', linewidth=0.5)

# Add buildings
# buildings = cartopy.feature.NaturalEarthFeature(
#             category='cultural',  name='buildings',
#             scale='10m', facecolor='none')
# ax.add_feature(buildings, edgecolor='black', zorder=10, linestyle = '-', linewidth=0.5)

ax.gridlines(draw_labels=True)
plt.show()
m
adapted_edges=[(u,v,k) for u,v,k,attr in graph_v.edges(keys=True, data=True) if 'osm_id' in attr and 'l3_adaptation' in attr['osm_id']]
print(adapted_edges)
# shortest_paths
# assets_in_sps2=shortest_paths_assets[('node_164', 'node_10409')]
assets_in_sps2=shortest_paths_assets[('node_164', 'node_11238')]
# shortest_paths[('node_164', 'node_11238')]
# assets_in_sps2
# [(u,v) for (u,v), (path, demand) in shortest_paths_assets.items() if '111997044' in path]
disrupted_shortest_paths['flood_DERP_RW_L_4326_2080430320'][('node_164', 'node_11238')]==shortest_paths[('node_164', 'node_11238')]
print('shortest_paths: ', shortest_paths[('node_164', 'node_11238')])
print('disrupted_shortest_paths: ', disrupted_shortest_paths['flood_DERP_RW_L_4326_2080430320'][('node_164', 'node_11238')])





basins_path = gpd.read_file(data_path / r'Floods\basins\hybas_eu_lev01-12_v1c\hybas_eu_lev08_v1c_valid.shp')
regions_path = gpd.read_file(data_path / r'QGIS_data\rhineland_palatinate.geojson')

basin_list_tributaries, basin_list_full_flood = find_basin_lists(basins_path, regions_path)


#!pip install seaborn
#PLOT INDIRECT DAMAGES AND DIRECT DAMGES FOR EACH HAZARD MAP SORTED BY SUMMED ADAPTATION COSTS
import matplotlib.pyplot as plt
import seaborn as sns
total_damages_adapted_df_mill=total_damages_adapted_df_mill.sort_values(by='summed_adaptation_costs')
fig, ax = plt.subplots(figsize=(10, 6))
sns.set_theme(style="whitegrid")
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.barplot(x='summed_adaptation_costs', y='hazard_map', data=total_damages_adapted_df_mill, palette='viridis', ax=ax)
ax.set_xlabel('Summed adaptation costs [M€]')
ax.set_ylabel('Hazard map')
plt.show()
# indirect damages adapted vs baseline
fig, ax = plt.subplots(figsize=(10, 6))
sns.set_theme(style="whitegrid")
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.barplot(x='indirect damage adapted [M€]', y='hazard_map', data=total_damages_adapted_df_mill, palette='viridis', ax=ax)
ax.set_xlabel('Indirect damage adapted [M€]')
ax.set_ylabel('Hazard map')
plt.show()

#!pip install seaborn
#adaptation cost for each basin/hazard map
import matplotlib.pyplot as plt
import seaborn as sns
total_damages_adapted_df_mill=total_damages_adapted_df_mill.sort_values(by='summed_adaptation_costs')
fig, ax = plt.subplots(figsize=(10, 6))
sns.set_theme(style="whitegrid")
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.barplot(x='summed_adaptation_costs', y='hazard_map', data=total_damages_adapted_df_mill, palette='viridis', ax=ax)
ax.set_xlabel('Summed adaptation costs [M€]')
ax.set_ylabel('Hazard map')
plt.show()